## 以 ResNet 18 为例

首先加载训练好的模型参数：

In [ ]:
resnet18 = models.resnet18()

# 修改全连接层的输出
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 2)

# 加载模型参数
checkpoint = torch.load(m_path)
resnet18.load_state_dict(checkpoint['model_state_dict'])

然后比较重要的是把模型放到 GPU 上，并且转换到eval模式：

In [ ]:
resnet18.to(device)
resnet18.eval()

In [ ]:
with torch.no_grad():
    for idx, img_name in enumerate(img_names):

        path_img = os.path.join(img_dir, img_name)

        # step 1/4 : path --> img
        img_rgb = Image.open(path_img).convert('RGB')

        # step 2/4 : img --> tensor
        img_tensor = img_transform(img_rgb, inference_transform)
        img_tensor.unsqueeze_(0)
        img_tensor = img_tensor.to(device)

        # step 3/4 : tensor --> vector
        outputs = resnet18(img_tensor)

        # step 4/4 : get label
        _, pred_int = torch.max(outputs.data, 1)
        pred_str = classes[int(pred_int)]

- 总结一下 inference 阶段需要注意的事项：
    - 确保 model 处于 eval 状态，而非 trainning 状态
    - 设置 torch.no_grad()，减少内存消耗，加快运算速度
    - 数据预处理需要保持一致，比如 RGB 或者 rBGR